<a href="https://colab.research.google.com/github/chrober24/A.UD_289.84_seminar/blob/main/StyleCLIP_playground_Custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text-Guided Editing of Images (Using CLIP and StyleGAN)

# Setup

In [ ]:
#@title Check GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-df3fccac-d004-721d-b056-bbcf7fd06b34)


In [ ]:
#@title Mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#@title Setup
import os

!git clone https://github.com/dvschultz/stylegan2-ada-pytorch/
!git clone https://github.com/rosinality/stylegan2-pytorch
!pip install ninja

!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

if os.path.isdir("/content/drive/MyDrive/Style_CLIP"):
  %cd "/content/drive/MyDrive/Style_CLIP/StyleCLIP"
elif os.path.isdir("/content/drive/"):
  %cd "/content/drive/MyDrive/"
  !mkdir Style_CLIP
  %cd Style_CLIP
  !git clone https://github.com/dvschultz/StyleCLIP.git
  %cd StyleCLIP
  !mkdir pkls
  !mkdir pts
else:
  !git clone https://github.com/dvschultz/StyleCLIP.git
  %cd StyleCLIP
  !mkdir pkls
  !mkdir pts

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 533, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 533 (delta 2), reused 0 (delta 0), pack-reused 524
Receiving objects: 100% (533/533), 8.42 MiB | 11.02 MiB/s, done.
Resolving deltas: 100% (303/303), done.
Cloning into 'stylegan2-pytorch'...
remote: Enumerating objects: 395, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 395 (delta 0), reused 1 (delta 0), pack-reused 392
Receiving objects: 100% (395/395), 122.52 MiB | 19.29 MiB/s, done.
Resolving deltas: 100% (200/200), done.
     |████████████████████████████████| 108 kB 23.4 MB/s 
     |████████████████████████████████| 53 kB 1.9 MB/s 
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-fngoyrm2
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-fngoyrm2
  Created wheel for clip: filename=c

# Model Conversion

This model requires a model file in the Rosinality format. You must convert your Pkl file to Pt format for this to run. You can do this with the next cell. Alternatively, you can use [this notebook](https://colab.research.google.com/github/dvschultz/stylegan2-ada-pytorch/blob/main/SG2_ADA_PT_to_Rosinality.ipynb)

In [ ]:
#@title Convert Pkl to Pt

%cd /content/stylegan2-ada-pytorch/

pkl_filepath = '/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/forest_gothic.pkl' #@param {type: 'string'}
pt_filepath = '/content/drive/MyDrive/Style_CLIP/StyleCLIP/pts/forest_gothic.pt' #@param {type: 'string'}

!python export_weights.py $pkl_filepath $pt_filepath

%cd /content/drive/MyDrive/Style_CLIP/StyleCLIP

# Settings

In [ ]:
experiment_type = 'free_generation' 

model_path = '/content/drive/MyDrive/Style_CLIP/StyleCLIP/pts/forest_gothic.pt' #@param {type:"string"}

description = 'A gothic cathedral full of trees' #@param {type:"string"}

latent_path = None 

optimization_steps = 500 #@param {type:"number"}

l2_lambda = 0.008

stylegan_size = "512" #@param ['256', '512', '1024']

learning_rate = 0.2 #@param {type: 'slider', min: 0.01, max: 1, step: 0.01}

truncation_psi = 0.7 #@param {type: 'slider', min:0, max:1, step:0.1}

create_video = True #@param {type:"boolean"}

args = {
    "description": description,
    "ckpt": model_path,
    "stylegan_size": int(stylegan_size),
    "lr_rampup": 0.05,
    "lr": learning_rate, #default 0.1
    "step": optimization_steps,
    "mode": experiment_type,
    "l2_lambda": l2_lambda,
    "latent_path": latent_path,
    "truncation": truncation_psi,
    "save_intermediate_image_every": 1 if create_video else 20,
    "results_dir": "results",
    "save_vector": True
}

# Run

In [ ]:
#@title Run Optimization


from optimization.run_optimization import main
from argparse import Namespace
result = main(Namespace(**args))

In [ ]:
#@title Visualize Result
from torchvision.utils import make_grid
from torchvision.transforms import ToPILImage
result_image = ToPILImage()(make_grid(result.detach().cpu(), normalize=True, scale_each=True, range=(-1, 1), padding=0))
h, w = result_image.size
result_image.resize((h // 2, w // 2))

In [ ]:
#@title Create Video

video_name = 'test' #@param {type: 'string'}
vid_path = './' + video_name + '.mp4'

!ffmpeg -r 15 -i ./results/%05d.png -c:v libx264 -vf fps=25 -pix_fmt yuv420p $vid_path